Connected to Python 3.9.13

In [ ]:
import firebase_admin
from firebase_admin import credentials, firestore
from datetime import datetime
from typing import List, Dict, Optional 

cred = credentials.Certificate("my-small-farm-system-firebase-adminsdk-fbsvc-343c1b46e5.json")
firebase_admin.initialize_app(cred)
db = firestore.client()

class FarmManager:
    def __init__(self, credential_path: str = None):
        """
        เริ่มต้นการเชื่อมต่อกับ Firebase Firestore
        
        Args:
            credential_path (str): path to Firebase service account key JSON file
        """
        if not firebase_admin._apps:
            if credential_path:
                cred = credentials.Certificate(credential_path)
            else:
                # ถ้าไม่ได้ระบุ path จะใช้ default credentials (สำหรับ environments เช่น Cloud Functions)
                cred = credentials.ApplicationDefault()
                
            firebase_admin.initialize_app(cred)
        
        self.db = firestore.client()
        self.collection_farms = 'farms'
        self.collection_users = 'users'
    
    def create_farm(self, user_id: str, farm_data: Dict) -> Dict:
        """
        สร้างฟาร์มใหม่และกำหนดให้ user เป็น owner
        
        Args:
            user_id (str): ID ของ user ที่สร้างฟาร์ม
            farm_data (Dict): ข้อมูลฟาร์ม เช่น name, description, location
            
        Returns:
            Dict: ข้อมูลฟาร์มที่สร้างเสร็จแล้วพร้อม ID
        """
        try:
            # ตรวจสอบว่าผู้ใช้มีอยู่จริง
            user_ref = self.db.collection(self.collection_users).document(user_id)
            if not user_ref.get().exists:
                raise ValueError(f"User with ID {user_id} does not exist")
            
            # เพิ่มข้อมูล meta
            farm_data['created_at'] = datetime.now()
            farm_data['updated_at'] = datetime.now()
            farm_data['owner_id'] = user_id
            farm_data['members'] = {user_id: 'owner'}  # เจ้าของฟาร์ม
            
            # บันทึกลง Firestore
            farm_ref = self.db.collection(self.collection_farms).document()
            farm_ref.set(farm_data)
            
            # อัปเดตข้อมูลผู้ใช้เพื่อเพิ่มฟาร์มนี้ในรายการฟาร์มของผู้ใช้
            user_ref.update({
                'farms': firestore.ArrayUnion([farm_ref.id])
            })
            
            return {"id": farm_ref.id, **farm_data}
            
        except Exception as e:
            print(f"Error creating farm: {e}")
            raise
    
    def add_member_to_farm(self, farm_id: str, user_id: str, role: str, invited_by: str) -> bool:
        """
        เพิ่มสมาชิกเข้าไปในฟาร์ม
        
        Args:
            farm_id (str): ID ของฟาร์ม
            user_id (str): ID ของ user ที่จะเพิ่ม
            role (str): สิทธิ์ในฟาร์ม (owner, manager, worker, etc.)
            invited_by (str): ID ของ user ที่ทำการ invite
            
        Returns:
            bool: True หากสำเร็จ, False หากไม่สำเร็จ
        """
        try:
            farm_ref = self.db.collection(self.collection_farms).document(farm_id)
            farm_data = farm_ref.get().to_dict()
            
            if not farm_data:
                raise ValueError(f"Farm with ID {farm_id} does not exist")
            
            # ตรวจสอบสิทธิ์: เฉพาะ owner หรือ manager สามารถเพิ่มสมาชิกได้
            if invited_by not in farm_data['members'] or farm_data['members'][invited_by] not in ['owner', 'manager']:
                raise PermissionError("You don't have permission to add members")
            
            # ตรวจสอบว่าผู้ใช้มีอยู่จริง
            user_ref = self.db.collection(self.collection_users).document(user_id)
            if not user_ref.get().exists:
                raise ValueError(f"User with ID {user_id} does not exist")
            
            # อัปเดตสมาชิกในฟาร์ม
            farm_ref.update({
                f'members.{user_id}': role,
                'updated_at': datetime.now()
            })
            
            # อัปเดตฟาร์มในรายการฟาร์มของผู้ใช้
            user_ref.update({
                'farms': firestore.ArrayUnion([farm_id])
            })
            
            return True
            
        except Exception as e:
            print(f"Error adding member to farm: {e}")
            return False
    
    def get_user_farms(self, user_id: str) -> List[Dict]:
        """
        ดึงข้อมูลฟาร์มทั้งหมดที่ user เป็นสมาชิกอยู่
        
        Args:
            user_id (str): ID ของ user
            
        Returns:
            List[Dict]: รายการฟาร์มที่ user เป็นสมาชิก
        """
        try:
            # ดึงข้อมูลผู้ใช้เพื่อรับรายการฟาร์ม
            user_ref = self.db.collection(self.collection_users).document(user_id)
            user_data = user_ref.get().to_dict()
            
            if not user_data or 'farms' not in user_data:
                return []
            
            farms = []
            for farm_id in user_data['farms']:
                farm_ref = self.db.collection(self.collection_farms).document(farm_id)
                farm_data = farm_ref.get().to_dict()
                
                if farm_data:
                    # เพิ่ม role ของ user ในฟาร์มนี้
                    farm_data['user_role'] = farm_data['members'].get(user_id, 'unknown')
                    farm_data['id'] = farm_id
                    farms.append(farm_data)
            
            return farms
            
        except Exception as e:
            print(f"Error getting user farms: {e}")
            return []
    
    def update_farm(self, farm_id: str, updates: Dict, user_id: str) -> bool:
        """
        อัปเดตข้อมูลฟาร์ม
        
        Args:
            farm_id (str): ID ของฟาร์ม
            updates (Dict): ข้อมูลที่ต้องการอัปเดต
            user_id (str): ID ของ user ที่ต้องการอัปเดต
            
        Returns:
            bool: True หากสำเร็จ, False หากไม่สำเร็จ
        """
        try:
            farm_ref = self.db.collection(self.collection_farms).document(farm_id)
            farm_data = farm_ref.get().to_dict()
            
            if not farm_data:
                raise ValueError(f"Farm with ID {farm_id} does not exist")
            
            # ตรวจสอบสิทธิ์: เฉพาะ owner หรือ manager สามารถอัปเดตฟาร์มได้
            if user_id not in farm_data['members'] or farm_data['members'][user_id] not in ['owner', 'manager']:
                raise PermissionError("You don't have permission to update this farm")
            
            # อัปเดตข้อมูล
            updates['updated_at'] = datetime.now()
            farm_ref.update(updates)
            
            return True
            
        except Exception as e:
            print(f"Error updating farm: {e}")
            return False
    
    def remove_member_from_farm(self, farm_id: str, user_id_to_remove: str, requested_by: str) -> bool:
        """
        ลบสมาชิกออกจากฟาร์ม
        
        Args:
            farm_id (str): ID ของฟาร์ม
            user_id_to_remove (str): ID ของ user ที่จะลบ
            requested_by (str): ID ของ user ที่ขอให้ลบ
            
        Returns:
            bool: True หากสำเร็จ, False หากไม่สำเร็จ
        """
        try:
            farm_ref = self.db.collection(self.collection_farms).document(farm_id)
            farm_data = farm_ref.get().to_dict()
            
            if not farm_data:
                raise ValueError(f"Farm with ID {farm_id} does not exist")
            
            # ตรวจสอบสิทธิ์: เฉพาะ owner หรือ manager สามารถลบสมาชิกได้
            if requested_by not in farm_data['members'] or farm_data['members'][requested_by] not in ['owner', 'manager']:
                raise PermissionError("You don't have permission to remove members")
            
            # ไม่สามารถลบ owner ออกได้
            if farm_data['members'].get(user_id_to_remove) == 'owner':
                raise PermissionError("Cannot remove farm owner")
            
            # อัปเดตสมาชิกในฟาร์ม
            farm_ref.update({
                f'members.{user_id_to_remove}': firestore.DELETE_FIELD,
                'updated_at': datetime.now()
            })
            
            # อัปเดตฟาร์มในรายการฟาร์มของผู้ใช้
            user_ref = self.db.collection(self.collection_users).document(user_id_to_remove)
            user_ref.update({
                'farms': firestore.ArrayRemove([farm_id])
            })
            
            return True
            
        except Exception as e:
            print(f"Error removing member from farm: {e}")
            return False

# ตัวอย่างการใช้งาน
if __name__ == "__main__":
    # เริ่มต้น FarmManager
    farm_manager = FarmManager(credential_path="path/to/serviceAccountKey.json")
    
    # สร้างฟาร์มใหม่
    new_farm = {
        "name": "ฟาร์มตัวอย่าง",
        "description": "นี่คือฟาร์มตัวอย่างสำหรับการทดสอบ",
        "location": "เชียงใหม่, ประเทศไทย",
        "type": "ผักออร์แกนิค"
    }
    
    # สมมติว่า user_id ของผู้ใช้ที่สร้างฟาร์ม
    user_id = "example_user_id"
    
    try:
        # สร้างฟาร์มใหม่
        created_farm = farm_manager.create_farm(user_id, new_farm)
        print(f"สร้างฟาร์มสำเร็จ: {created_farm['id']}")
        
        # เพิ่มสมาชิกใหม่
        new_user_id = "new_user_id"
        success = farm_manager.add_member_to_farm(
            created_farm['id'], 
            new_user_id, 
            "worker", 
            user_id
        )
        print(f"เพิ่มสมาชิกสำเร็จ: {success}")
        
        # ดึงฟาร์มทั้งหมดของผู้ใช้
        user_farms = farm_manager.get_user_farms(user_id)
        print(f"ฟาร์มทั้งหมดของผู้ใช้: {len(user_farms)} ฟาร์ม")
        
        # อัปเดตข้อมูลฟาร์ม
        updates = {"description": "อัพเดตคำอธิบายฟาร์ม"}
        update_success = farm_manager.update_farm(created_farm['id'], updates, user_id)
        print(f"อัปเดตฟาร์มสำเร็จ: {update_success}")
        
    except Exception as e:
        print(f"เกิดข้อผิดพลาด: {e}")

FileNotFoundError: [Errno 2] No such file or directory: 'my-small-farm-system-firebase-adminsdk-fbsvc-343c1b46e5.json'